In [1]:
import os
from PIL import Image
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import tqdm

class CustomDatasetYOLO:
    def __init__(self, folder_path, image_size=416, transforms=None):
        self.folder_path = folder_path
        self.transforms = transforms
        self.image_size = image_size
        self.data_pairs = self._load_data_pairs()

    def _load_data_pairs(self):
        image_files = []
        json_files = {}
        data_pairs = []

        # 폴더를 재귀적으로 검색하여 이미지 파일과 JSON 파일 목록을 생성
        for root, _, files in os.walk(self.folder_path):
            for file in files:
                if file.endswith((".jpg", ".jpeg", ".png")):
                    image_files.append(os.path.join(root, file))
                elif file.endswith(".json"):
                    json_files[os.path.splitext(file)[0]] = os.path.join(root, file)

        # 진행률 표시줄 추가
        for image_file_path in tqdm.tqdm(image_files, desc="Loading data pairs"):
            # 이미지 파일 이름에서 확장자를 제외한 부분 가져오기
            image_name = os.path.splitext(os.path.basename(image_file_path))[0]

            # 해당 이미지 파일과 매칭되는 JSON 파일 찾기
            if image_name in json_files:
                json_file_path = json_files[image_name]
                data_pairs.append((image_file_path, json_file_path))
            else:
                print(f"JSON file not found for image: {image_file_path}")

        return data_pairs

    def resize_image_and_annotations(self, image_path, annotations, categories, target_size):
        img = Image.open(image_path)
        img_resized = img.resize(target_size, Image.LANCZOS)  # 이미지 크기 조정

        width_ratio = target_size[0] / img.width  # 너비 비율 계산
        height_ratio = target_size[1] / img.height  # 높이 비율 계산

        # 한글에서 영어로 라벨을 매핑하는 딕셔너리 (예시)
        label_mapping = {'화방': 1, '줄기': 2, '잎': 3, '열매': 0}

        labels = []
        for ann in annotations:
            x, y, w, h = ann['bbox']
            category_id = ann['category_id']
            category_name = categories.get(category_id, '')

            # 라벨 매핑
            label = label_mapping.get(category_name, -1)
            if label == -1:
                # 처리할 수 없는 경우 스킵
                continue

            # 조정된 이미지의 바운딩 박스 좌표 계산
            x_resized = x * width_ratio
            y_resized = y * height_ratio
            w_resized = w * width_ratio
            h_resized = h * height_ratio

            # YOLO 형식의 정규화된 좌표 계산
            x_center = (x_resized + w_resized / 2) / target_size[0]
            y_center = (y_resized + h_resized / 2) / target_size[1]
            box_width = w_resized / target_size[0]
            box_height = h_resized / target_size[1]

            # YOLO 형식 문자열로 변환
            label_str = f"{label} {x_center} {y_center} {box_width} {box_height}"
            labels.append(label_str)

        return img_resized, labels

    def create_label_files(self):
        def process_image(image_path, json_path, split):
            parent_folder = os.path.join(self.folder_path, split)
            images_folder = os.path.join(parent_folder, 'images')
            labels_folder = os.path.join(parent_folder, 'labels')
            os.makedirs(images_folder, exist_ok=True)
            os.makedirs(labels_folder, exist_ok=True)

            img_name = os.path.splitext(os.path.basename(image_path))[0]
            txt_file = os.path.join(labels_folder, f"{img_name}.txt")
            img_resized_path = os.path.join(images_folder, f"{img_name}.jpg")

            with open(json_path, 'r') as json_file:
                json_data = json.load(json_file)

            # 라벨이 없는 경우 패스
            if not json_data.get('annotations'):
                print(f"No annotations found for {image_path}, skipping...")
                return

            # categories 정보를 가져옴
            categories = {cat['id']: cat['name'] for cat in json_data['categories']}

            img_resized, labels = self.resize_image_and_annotations(image_path, json_data['annotations'], categories, (self.image_size, self.image_size))

            img_resized.save(img_resized_path)

            with open(txt_file, 'w') as f:
                for label in labels:
                    f.write(f"{label}\n")

        with ThreadPoolExecutor() as executor:
            futures = []
            for image_path, json_path in self.data_pairs:
                split = image_path.split(os.sep)[-3]  # 'train', 'val', 'test'
                futures.append(executor.submit(process_image, image_path, json_path, split))

            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    print(f"Exception occurred: {e}")

        print("YOLO 라벨 파일 생성이 완료되었습니다.")


In [2]:
if __name__ == "__main__":
    dataset = CustomDatasetYOLO('./datasets', image_size=640)
    dataset.create_label_files()

Loading data pairs: 100%|██████████| 411/411 [00:00<00:00, 203501.23it/s]

YOLO 라벨 파일 생성이 완료되었습니다.


In [3]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
from ultralytics import YOLO
model = YOLO("yolov10n.yaml")
model.to(device)

YOLOv10(
  (model): YOLOv10DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track

In [5]:


model.train(data="./farmpj.yaml", epochs=30, imgsz=640,  workers=0)
print(model)

New https://pypi.org/project/ultralytics/8.2.42 available 😃 Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov10n.yaml, data=./farmpj.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=cuda:0, workers=0, project=None, name=train24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

train: Scanning C:\Users\user\Documents\GitHub\model_dev\datasets\train\labels... 275 images, 0 backgrounds, 0 corrupt: 100%|██████████| 275/275 [00:00<00:00, 648.79it/s]

train: New cache created: C:\Users\user\Documents\GitHub\model_dev\datasets\train\labels.cache



val: Scanning C:\Users\user\Documents\GitHub\model_dev\datasets\val\labels... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<00:00, 545.69it/s]

val: New cache created: C:\Users\user\Documents\GitHub\model_dev\datasets\val\labels.cache


...

      30/30      3.18G      1.673      1.961      2.314      1.451      3.327      2.155          5        640: 100%|██████████| 18/18 [00:06<00:00,  2.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.84it/s]

                   all         80        118      0.661      0.402      0.421      0.243



30 epochs completed in 0.078 hours.
Optimizer stripped from runs\detect\train24\weights\last.pt, 5.8MB
Optimizer stripped from runs\detect\train24\weights\best.pt, 5.8MB

Validating runs\detect\train24\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.9 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLOv10n summary (fused): 285 layers, 2695976 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.12it/s]


                   all         80        118       0.66      0.401      0.422      0.243
                  buds         80         34       0.49      0.593      0.614      0.295
                  stem         80         25          1          0     0.0292     0.0105
                  leaf         80         59      0.491       0.61      0.622      0.423
Speed: 0.4ms preprocess, 4.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\detect\train24
YOLOv10(
  (model): YOLOv10DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    